In [60]:
#  CONFIGURACIÓN INICIAL


from IPython.display import display  # Para mostrar imágenes en Colab
import matplotlib.pyplot as plt      # Para hacer gráficos y comparaciones
from PIL import Image, ImageFilter   # Librería para trabajar con imágenes
import requests                      # Para descargar imágenes de URLs
from io import BytesIO              # Para manejar imágenes en memoria
import numpy as np                   # Para operaciones matemáticas con arrays
import time                         # Para el delay en el menú

from google.colab import drive
drive.mount('/content/drive')  # Conecta con el Google Drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **PRIMERA PARTE**

## 1) Funcion para leer y redimenzionar segun la plataforma

In [61]:

def redimensionar(path, plataforma, mostrar=False):
    """
    Esta función toma una imagen y la ajusta al tamaño de cada red social.

    - path: puede ser una ruta local ("/content/drive/...") o una URL
    - plataforma: instagram, youtube, twitter o facebook
    - mostrar: si querés ver la imagen o no
    """

    # Diccionario con los tamaños recomendados de cada red social
    tamaños = {
        "instagram": (1080, 1080),
        "youtube": (1280, 720),
        "twitter": (1600, 900),
        "facebook": (1200, 630)
    }

    plataforma = plataforma.lower()

    # Validación: si la plataforma no existe
    if plataforma not in tamaños:
        raise ValueError("Plataforma no válida.")

    w_d, h_d = tamaños[plataforma]

    # ver si es url o archivo local
    if path.startswith("http"):
        # Si es URL, descargamos la imagen
        respuesta = requests.get(path)
        img = Image.open(BytesIO(respuesta.content))
    else:
        # Si no, abrimos desde el archivo local
        img = Image.open(path)


    img.thumbnail((w_d, h_d))

    # Creamos un fondo negro para ajustar el tamaño y no estirar la imagen
    fondo = Image.new("RGB", (w_d, h_d), (0, 0, 0))

    # Centramos la imagen en el fondo
    x = (w_d - img.width) // 2
    y = (h_d - img.height) // 2
    fondo.paste(img, (x, y))

    if mostrar:
        display(fondo)

    return fondo

## Ecualizacion del histograma (ajustar contraste)

In [62]:
def ecualizar_contraste(img, mostrar=False, guardar=False):
    """
    Mejora el contraste de fotos oscuras o con exceso de luz.
    Usa el histograma para redistribuir los tonos de gris.
    """

    if isinstance(img, str):  # Si pasaron una ruta en vez de una imagen
        img = Image.open(img)

    img_original = img.copy()  # Guardamos una copia de la original para comparar

    # Convertimos a escala de grises (blanco y negro)
    img_gray = img.convert("L")
    arr = np.array(img_gray)  # Convertimos a array de numpy para hacer cálculos

    # Calculamos el histograma (cuántos píxeles hay de cada tono)
    hist, bins = np.histogram(arr.flatten(), 256, [0, 256])

    # CDF = Función de Distribución Acumulativa
    cdf = hist.cumsum()  # Suma acumulada del histograma
    cdf_norm = 255 * cdf / cdf[-1]  # Normalizamos entre 0 y 255

    # Interpolamos para ecualizar
    arr_eq = np.interp(arr.flatten(), bins[:-1], cdf_norm)
    arr_eq = arr_eq.reshape(arr.shape).astype("uint8")
    eq_img = Image.fromarray(arr_eq)  # Convertimos de vuelta a imagen

    if mostrar:
        # Creamos una figura con 2 sub-gráficos (lado a lado)
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        # Izquierda: Original (A COLOR)
        axes[0].imshow(img_original)
        axes[0].set_title('Original', fontsize=12)
        axes[0].axis('off')

        # Derecha: Ecualizada (Blanco y negro con mejor contraste)
        axes[1].imshow(eq_img, cmap='gray')
        axes[1].set_title('Ecualizada', fontsize=12)
        axes[1].axis('off')

        plt.tight_layout()  # Ajusta el espaciado

        if guardar:
            plt.savefig('comparacion_ecualizacion.jpg', dpi=150)
            print("Figura guardada como 'comparacion_ecualizacion.jpg'")

        plt.show()

    return eq_img


## Filtros


In [63]:
# Diccionario con los filtros de Pillow
filtros_pillow = {
    "blur": ImageFilter.BLUR,           # Desenfoque
    "contour": ImageFilter.CONTOUR,     # Solo contornos
    "detail": ImageFilter.DETAIL,       # Resalta detalles
    "sharpen": ImageFilter.SHARPEN      # Nitidez
}

def aplicar_filtro_obj(img, filtro, mostrar=False):
    """
    Aplica un filtro de Pillow a la imagen.
    """


    if filtro not in filtros_pillow:
        raise ValueError("Filtro no válido.")


    img_filtrada = img.filter(filtros_pillow[filtro])

    if mostrar:
        display(img_filtrada)

    return img_filtrada

def dodge(front, back):
    """
    Técnica de edición fotográfica que aclara las zonas oscuras.
    Se usa para crear el efecto de boceto a lápiz.
    """
    front = np.array(front).astype('float')
    back = np.array(back).astype('float')


    result = front * 255 / (255 - back + 1e-6)  # 1e-6 para evitar división por cero
    result[result > 255] = 255  # Limitamos a 255

    return Image.fromarray(result.astype('uint8'))

## Funcion generar boceto


In [64]:
def generar_boceto_obj(img, persona=True, mostrar=False):
    """
    Convierte una foto en un boceto tipo lápiz.
    """

    # Validación: solo funciona si la IA detectó una persona
    if not persona:
        raise Exception("La IA no detectó una persona en la imagen.")

    # Convertimos a escala de grises
    img_gray = img.convert("L")

    # Aplicamos un blur gaussiano (desenfoque suave)
    blur = img_gray.filter(ImageFilter.GaussianBlur(5))

    # Aplicamos técnica dodge para el efecto boceto
    boceto = dodge(img_gray, blur)

    if mostrar:
        display(boceto)

    return boceto

## Menu interactivo con excepciones

In [65]:



def menu():
    """
    Menú principal de la aplicación.
    Valida que siempre se cargue primero una imagen antes de procesarla.
    """

    ultima_img = None    # Guarda la última imagen procesada
    original_img = None  # Guarda la imagen original (sin procesar)

    while True:  # Loop infinito hasta que el usuario elija salir
        time.sleep(1)
        # Espera 1 segundo antes de mostrar el menú, ya que colab muestra para
        # colocar el input (es un bug)

        # Mostramos las opciones
        print("""
1) Cargar + Redimensionar imagen
2) Ecualizar contraste
3) Aplicar filtro
4) Generar boceto
5) Restaurar imagen original
0) Salir
""")
        op = input("Opción: ")

        try:  # Manejo de errores con try-except

            # OPCIÓN 1: Cargar imagen
            if op == "1":
                path = input("Ruta o URL: ")
                plataforma = input("Plataforma (instagram/youtube/twitter/facebook): ")
                mostrar = input("¿Mostrar imagen redimensionada? (s/n): ").lower() == 's'

                ultima_img = redimensionar(path, plataforma)
                original_img = ultima_img.copy()  # Guardamos el original
                ultima_img.save("redimensionada.jpg")  # Guardamos la imagen redimensionada

                print("Imagen cargada y redimensionada.\n")
                if mostrar:
                    display(ultima_img)

            # OPCIÓN 2: Ecualizar contraste
            elif op == "2":

                if ultima_img is None:
                    print("Primero debe cargar una imagen.\n")
                    continue  # Vuelve al inicio del while

                mostrar = input("¿Mostrar comparación original vs ecualizada? (s/n): ").lower() == 's'
                guardar = input("¿Guardar la figura de comparación? (s/n): ").lower() == 's'

                ultima_img = ecualizar_contraste(ultima_img, mostrar, guardar)
                ultima_img.save("ecualizada.jpg")
                print("Contraste ecualizado.\n")

            # OPCIÓN 3: Aplicar filtro
            elif op == "3":
                if ultima_img is None:
                    print("Primero debe cargar una imagen.\n")
                    continue

                f = input("Filtro a aplicar (blur/contour/detail/sharpen): ").lower()
                mostrar = input("¿Mostrar imagen con filtro aplicado? (s/n): ").lower() == 's'

                ultima_img = aplicar_filtro_obj(ultima_img, f)
                ultima_img.save(f"filtro_{f}.jpg")  # Guarda con el nombre del filtro
                print("Filtro aplicado.\n")

                if mostrar:
                    display(ultima_img)

            # OPCIÓN 4: Generar boceto
            elif op == "4":
                if ultima_img is None:
                    print("Primero debe cargar una imagen.\n")
                    continue

                mostrar = input("¿Mostrar boceto generado? (s/n): ").lower() == 's'

                ultima_img = generar_boceto_obj(ultima_img)
                ultima_img.save("boceto.jpg")
                print("Boceto generado.\n")

                if mostrar:
                    display(ultima_img)

            # OPCIÓN 5: Restaurar original
            elif op == "5":
                if original_img is None:
                    print("Primero debe cargar una imagen.\n")
                else:
                    ultima_img = original_img.copy()  # Restauramos la original
                    print("Imagen restaurada al estado original.\n")

                    mostrar = input("¿Mostrar imagen restaurada? (s/n): ").lower() == 's'
                    if mostrar:
                        display(ultima_img)

            # OPCIÓN 0: Salir
            elif op == "0":
                print("Saliendo del programa.")
                break  # Rompe el while, termina el programa

            # Si ingresó un número que no está en el menú
            else:
                print("Opción inválida.\n")

        # Si hay algún error, lo mostramos pero no se cierra el programa
        except Exception as e:
            print("Error:", e, "\n")



menu()

Output hidden; open in https://colab.research.google.com to view.